# Challenge Problem 3: Genome Wide Association Study

## Background:

(Adapted From BMI 704, Dr. Chirag Patel)

Genome-wide association studies (GWASs) are arguably the most significant biomedical advance in the last decade. Utilizing an observational case-control design, populations with disease are analytically compared to those without to discern patterns in differences in genetic variant, or single nucleotide polymorphism (SNP),frequencies in each of the respective populations. If statistically differences in SNP frequencies in cases vs
controls are found, one can conclude the SNP locus may have a causal relationship, implicating a gene, genetic region, or gene regulator in the disease of interest.

Analyzing GWASs provide lessons in observational research, including statistical association and multiple testing. They also provide a way to create hypotheses about biological basis of disease for further study. We will execute a GWAS analysis using data from the Wellcome Trust Case-Control Consortium (WTCCC) on Type 2 Diabetes, Type 1 Diabetes, Bipolar Disorder, Rheumatoid Arthritis, Chron's Disease and Hypertension versus 1,500 controls.

Note: There are standard tools and pipelines available to do this but we will do it the old fashioned way!

The Data is provided in separate directories under the primary WTCCC directory. The subdirectories are labeled as such:
* 58C : control population 1
* NBS : control population 2
* RA : rheumatoid arthritis
* T2D : type 2 diabetes
* T1D : type 1 diabetes
* CAD : coronary artery disease
* HT : hypertension
* CD : Crohn’s disease
* BD : bipolar disorder

All of the GWAS data is broken down per chromosome, 1-22 plus X and is written into the file name, as well as the disease. For example:

Affx_gt_ 58C _Chiamo_ 06 .tped.gz

Contains GWAS data for the 6th (06) chromosome on control population 1 (58C). The biggest files are chromosome 1 and 2 and smallest is 22.


Each row is a SNP (there are 6207 SNPs measured in Chromosome 22). The first through fourth columns denote the chromosome number, the second is the WTCCC snp_id (not to be confused with the rsid), the start coordinate (0), and the end coordinate. Thereafter, each column is a genotype (where each base is separated by a space character) for that SNP for each individual.

Additionally, each subdirectory also contains a ‘snps_info.tar.gz’, which contains the snp_id and rsID for each SNP for each chromosome. These files can enable you to map the WTCCC snp_id to the rsid, which is helpful to find the closest genes to a SNP (for example when you query the GWAS catalog).

In [88]:
import pandas as pd
from os import path
import csv
import numpy as np
from time import time
!pwd

/Users/varun/Desktop/BeaverWorks/Week3Public/GWAS


In [222]:
## Load the test data 
t1d_22_file = '/Users/varun/Desktop/BeaverWorks/Week3Public/BWSI_100set/T2D/Affx_gt_T2D_Chiamo_22.tped.gz'
t1d = pd.read_csv(t1d_22_file, compression='gzip', header=None, sep='\t')

In [223]:
## Look at the shape of the data frame - you should have 6207 SNPs for 2003 individuals
t1d.shape

(100, 2003)

In [224]:
## Look at the top 10 rows
# t2d.head(10)
t1d_header = t1d.loc[:, :3]
t1d_body = t1d.loc[:,4:]
t1d_header.head()

,0,1,2,3
0,22,SNP_A-4234155,0,32056663
1,22,SNP_A-2223191,0,35653934
2,22,SNP_A-4202732,0,32956968
3,22,SNP_A-1811354,0,46997362
4,22,SNP_A-1797179,0,32955711


In [225]:
control_file = '/Users/varun/Desktop/BeaverWorks/Week3Public/BWSI_100set/58C/Affx_gt_58C_Chiamo_22.tped.gz'
control_58c = pd.read_csv(control_file, compression='gzip', header=None, sep='\t')

In [226]:
control_58c_header = control_58c.loc[:, :3]
control_58c_body = control_58c.loc[:, 4:]
control_58c_body.head()

,4,5,6,7,8,9,10,11,12,13,...,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507
0,C T,C T,C T,T T,C T,T T,C T,C T,T T,C T,...,T T,C C,C T,C T,C T,C C,C T,T T,C T,T T
1,T T,C C,C T,T T,C C,C T,C T,T T,T T,C C,...,C T,C T,C T,C T,T T,C T,C T,C C,T T,C C
2,G G,A G,A G,G G,A G,A G,G G,A G,A G,A G,...,A G,A G,A G,G G,G G,G G,G G,G G,A G,G G
3,A G,A A,A G,A A,A G,G G,A A,A A,A A,A A,...,A A,A G,A A,A A,A G,A G,A A,A A,A G,A A
4,G G,A G,A G,G G,A G,A G,G G,A G,A G,A G,...,A G,A G,A G,G G,G G,G G,G G,G G,A G,G G


In [227]:
control_file = '/Users/varun/Desktop/BeaverWorks/Week3Public/BWSI_100set/NBS/Affx_gt_NBS_Chiamo_01.tped.gz'
control_nbs = pd.read_csv(control_file, compression='gzip', header=None, sep='\t')

In [228]:
control_nbs_header = control_nbs.loc[:, :3]
control_nbs_body = control_nbs.loc[:, 4:]
control_nbs.head()

,0,1,2,3,4,5,6,7,8,9,...,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503
0,1,SNP_A-1938722,0,15385344,T T,T T,C T,C T,T T,C C,...,C T,C T,C C,C T,C T,T T,C T,C C,C T,C C
1,1,SNP_A-4217222,0,243847383,C T,C T,T T,C T,C T,C T,...,T T,C C,C T,T T,C T,C T,C T,C T,T T,T T
2,1,SNP_A-2023862,0,192847341,T T,T T,T T,T T,T T,T T,...,T T,T T,T T,T T,T T,T T,T T,T T,T T,T T
3,1,SNP_A-4196224,0,95890895,G G,G G,A G,A G,A G,G G,...,G G,G G,G G,G G,G G,G G,G G,A A,G G,G G
4,1,SNP_A-2286934,0,189084167,A A,A G,G G,G G,G G,A A,...,G G,G G,A G,A G,A G,A G,A G,A G,A G,G G


In [245]:
snp_file = '/Users/varun/Desktop/BeaverWorks/Week3Public/BWSI_100set/58C/snps/snps_01'
snp = pd.read_csv(snp_file, header=None, sep='\t')

combined = pd.concat([control_58c_header, snp], axis=1)
combined = combined[:100]

control_58c_header = control_58c_header.rename(columns={1:'SNP'})
snp = snp.loc[:, 3:]
snp = snp.rename(columns={3:'SNP'})
combined = control_58c_header.join(snp.set_index('SNP'), on='SNP')
combined.head()

,0,SNP,2,3,4
0,22,SNP_A-4234155,0,32056663,NaN
1,22,SNP_A-2223191,0,35653934,NaN
2,22,SNP_A-4202732,0,32956968,NaN
3,22,SNP_A-1811354,0,46997362,NaN
4,22,SNP_A-1797179,0,32955711,NaN


In [304]:
t1d_1 = t1d.loc[0, 4:]
control_58c_1 = control_58c_body.loc[0]
print("Type 2 Diabetes:")
print(computeAlleleFrequency(getAlleleCounts(t1d_1)[0], getAlleleCounts(t1d_1)[1]))
t1_minor = computeAlleleFrequency(getAlleleCounts(t1d_1)[0], getAlleleCounts(t1d_1)[1])
print(1-computeAlleleFrequency(getAlleleCounts(t1d_1)[0], getAlleleCounts(t1d_1)[1]))
t1_major = 1 - t1_minor
print("Control:")
print(computeAlleleFrequency(getAlleleCounts(control_58c_1)[0], getAlleleCounts(control_58c_1)[1]))
control_minor = computeAlleleFrequency(getAlleleCounts(control_58c_1)[0], getAlleleCounts(control_58c_1)[1])
print(1-computeAlleleFrequency(getAlleleCounts(control_58c_1)[0], getAlleleCounts(control_58c_1)[1]))
control_major = 1-control_minor
# print(control)
print()
print("Odds Ratio:")
print(computeOR([t1_minor, t1_major, control_minor, control_major]))

print("\nChi-Squared:")
print(getPValue([t1_minor, t1_major]))
print(getPValue([control_minor, control_major]))

print("\nHWE:")
print(HWEChiSq(t1_minor, [t1d_1.to_string().count('C C'), t1d_1.to_string().count('C T'), t1d_1.to_string().count('T T')]))
print(HWEChiSq(control_minor, [control_58c_1.to_string().count('C C'), control_58c_1.to_string().count('C T'), control_58c_1.to_string().count('T T')]))


Type 2 Diabetes:
0.38244122061030517
0.6175587793896948
Control:
0.38331117021276595
0.616688829787234

Odds Ratio:
0.9963249409766974

Chi-Squared:
0.8141174907213498
0.8154681593364395

HWE:
0.594335487291052
0.9149630590292988


In [212]:
## function to load the data file
def loadFile (disease, chrom_num):
    ## construct your file name based on the disease and chrom_number
    fileName = '/Users/varun/Desktop/BeaverWorks/Week3Public/BWSI_100set/' + str(disease) + '/Affx_gt_T1D_Chiamo_' + str(chrom_num) + '.tped.gz'
    dat = pd.read_csv(fileName, compression='gzip', header=None, sep='\t')
    return (dat)

In [195]:
## compute Allele Frequency for the minor allele
def computeAlleleFrequency (f_C, f_T):
    
    """
    f_C = minor allele count
    f_T = major allele count
    minor_allele_frequency = f_C/ (f_C+f_T) 
    """
    minor_allele_frequency = f_C/ (f_C+f_T) 
    return (minor_allele_frequency)



In [203]:
def getAlleleCounts (genotype):
    """
    genotype = "C C 	C T 	C T 	C T 	T T"
    Allele frequency:
    count the "Cs", and the "Ts"
    f_C = # of Cs
    f_T = # of T's
    """

    
    return np.array([genotype.str.count('C').sum(), genotype.str.count('T').sum()])
    

## Hardy Weinberg Equilibrium

In [303]:
## Function takes as input the minor allele frequency (p) and the population genotype counts
## returns the p-value from the chisq test
def HWEChiSq (p, populationGenotypeCounts):
    """
        |  CC | CT  | TT |
    BP  | 270 | 957 | 771|
    compute the HWE
      -- p = frequency of the C allele
      -- q = 1-p = frequency of the T allele
      -- if the population genotype is in Hardy Weinberg Equilibrium, we would expect the genotype frequencies to be
          CC = p^2*N 
          CT = 2pq*N 
          TT = q^2*N 
      -- to compute the deviation from HWE
         (observed - expected)^2 / expected
      -- do a chi.squared test to check if the deviation is significant
    """
    total = sum(populationGenotypeCounts)
        
    q = 1-p
    
    observed = populationGenotypeCounts[0]
    expected = p**2*total
    
    CC = (observed-expected)**2/expected
    
    
    observed = populationGenotypeCounts[1]
    expected = 2*p*q*total
    
    CT = (observed-expected)**2/expected
    
    observed = populationGenotypeCounts[2]
    expected = q**2*total
    
    TT = (observed-expected)**2/expected
    
    from scipy.stats import chisquare
    from scipy.stats.distributions import chi2
    return chi2.sf(CC+CT+TT, 1)
    

In [236]:
## Compute the Odds Ratio
## takes as input a 2X2  confusion matrix
## returns the odds ratio
def computeOR (confusionMatrix):
    ad = confusionMatrix[0] * confusionMatrix[3]
    bc = confusionMatrix[1] * confusionMatrix[2]
    return ad/bc

In [255]:
## Execute a chisq test to determine if the odds ratio is significant
## return the p-value from the chisq test
def getPValue (confusionMatrix):
    from scipy.stats import chisquare
    return chisquare(confusionMatrix).pvalue

In [15]:
## plot the data
## manhattan plot
## the X axis represents the chromosomes 
## the y-axis is the -log(p.value)
def manhattanPlot (diseaseResults):
    """
    group data by chromosomes, SNP before plotting
    """
    return 

In [17]:
## Write code to run the GWAS

## Write a function that will take the following inputs:
## (a) control files - 58C and NBS
## (b) the disease - T2D, T1D, HT, BD, CD, RA
## (c) the chromosome number

## Your function should output a csv file with the following information
## SNP Id, RSID, CHROMOSOME NUMBER, MINOR ALLELE, MAJOR ALLELE, MINOR ALLELE FREQUENCY, MAJOR ALLELE FREQUENCY, 
## ODDS RATIO (MINOR vs MAJOR ALLELE), P-Value for ODDS RATIO, Chi square for the ODDS RATIO, HWE deviation P-VALUE
## 

def gwas (controlFile, disease, chrom_num):
    """
    1. Load the control file for the specified chromosome number. Since there are two control populations, 
       it would be best to combine the two controls together
    2. Load the disease file for the given chromosome number
    3. Split the header from the SNP data for the controls and the disease
    3. Load the SNP file -- this gives you the mapping between the WTCCC SNP ids and the RSID
    4. For each SNP in the disease file and its matching SNP in the control file:
       -- to make it easier, before you begin do a check for zygosity. If either the controls or the disease 
       are homozygous, skip the SNP
       genotype CC, TT : skip
       genotype CC CT TT : run the following analysis
       a. get the minor and major allele counts and compute the minor allele frequency for the disease population
       b. get the minor and major allele counts and compute the minor allele frequency for the control population
       c. conduct the allelic test -- compute the odds ratios to test the frequency of the minor allele 
          in the disease population compared to the control population
          -- you can build a confusion matrix based on the counts that you have calculated in a and b
          -- compute your odds ratios based on the minor allele of the control population
          
          |                  | C    | T    | 
          |------------------+ ---  + ---  +
          | Bipolar Disorder | 1529 | 2469 |
          |------------------+----  + ---  +
          | Healthy Controls | 2270 | 3738 |
          |------------------+----- +----- +
          
          (Test : your OR for above should be :1.019)
       d. Do a chi square test to test the significant and record the p-value
       d. for each SNP calculate the deviation from Hardy-Weinberg equilibrium
       e. create an output row containing:
       SNP Id, RSID, CHROMOSOME NUMBER, MINOR ALLELE, MAJOR ALLELE, MINOR ALLELE FREQUENCY, 
       MAJOR ALLELE FREQUENCY, ODDS RATIO (MINOR vs MAJOR ALLELE), 
       P-Value for ODDS RATIO, CHI_SQ for the ODDS RATIO, HWE deviation P-VALUE  
    5. Save the result file for the chromosome
    """
    
    data = loadFile(disease, chrom_num)
    
    
    


In [18]:
## Write a main function that will invoke gwas for all chromosomes (1 through 22, don't include the X chromosome)

def runGWAS (disease):
    """
    1. For each chromosome 1:22 call the gwas function
    2. Merge all the result files into one 
    3. Filter your data
       a. remove all SNPs that deviate signficantly from HWE equilibrium -- these may indicate an error or 
          population specific deviation. For this remove all SNPs for which HWE deviation P-Value < 0.05
       b. remove all SNPs that have minor allele frequency less than 1% -- not enough data
    4. Draw a manhattan plot
       c. draw a line at your Bonferroni threshold (Bonferroni correction): 
       if your significance is set at p.value < 0.05, then Bonferroni correction = 0.05/number of tests that you ran
    """

<img src="manhattan.png">

In [ ]:
## How many SNPs did you identify that exceeded the Bonferroni-level of significance?

## How much can you trust your results

Typically, you want to examine the genome-wide distribution of your chi_square distribution against an expected null distribution. Visually you can do this using a quantile-quantile plot. Significant deviation from the null distribution may point to population stratification, familial relationships, technical bias, poor sample collection or sometimes even undetected sample duplications.
(Slide courtesy - Dr. Chirag Patel)

<img src="qqplot_example.png">


In [20]:
## Based on your results do a quantile-quantile plot for your data

##load your results 
##sort/order the -log10 of your p values in increasing order
##generate an expected set of -log10(p_values)
##plot the expected values on the x-axis and the observed -log10 (pvalues) on the y axis
##Alternatively use: p = observed p values
##stats.probplot(p, dist="norm", plot=pylab)
##pylab.show()


### Genomic Inflation Factor

Mathematically, you can correct for some of the deviation by computing an inflation factor and then correcting for this inflation factor. This quantity is termed as genomic inflation factor ($\lambda$)

$\lambda$ = observed median of test statistic distribution / expected median of the test statistic 
distribution.  For 1 degree of freedom, the X2 distribution has an expected median of 0.455

$\lambda$ <= 1.05 is considered acceptable. >1.1 is troubling, and indicates there is some inflation of the p values.


In [ ]:
## Compute the genomic inflation factor

##labmda =  median(CHI2)/qchisq(0.5, 1)
##if (gif > 1.05), correct your computed Chi.square values and recompute the p_values
##chi2_corrected = CHI2/lambda, 
##pval_corrected = pchisq(chi2_corrected, 1, lower.tail = F)
##re-draw the qqplot using the corrected pvalues (pval_corrected)


## What does this mean?

Identify a SNP that is significant (above the Bonferroni threshold). 
What is the p-value of association and odds ratio for the SNP?
What is the interpretation of the odds ratio? 
What gene is this SNP associated with and its putative function in your disease pathogenesis? 


## Polygenic Risk Scores (PRS)

A PRS is the cumulative risk of all SNP for a single patient. For example, if a patient has a 2 snps with OR=2 each, then their PRS is 4. The underlying assumption here is that each allele has an equal contribution and there is no interation between SNPS, that is the SNPS are independent. 

$prs =\sum_{i=1}^{m}-log_{10}(x_i) \cdot n_i$